<a href="https://colab.research.google.com/github/Hani1-2/Arabic-Speech-Recognizer/blob/main/ANN_arabic_speech_recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import json
data_path = '/content/drive/MyDrive/FYP-Code/data_prep_ann.json'
with open(data_path, "r") as fp:
      data = json.load(fp)


In [2]:
import numpy as np


In [40]:
### Split the dataset into independent and dependent dataset
X=np.array(data['MFCCs'])
y=np.array(data['labels'])

In [41]:
X.shape

(21952, 40)

In [42]:
y.shape

(21952,)

In [43]:
### Label Encoding
###y=np.array(pd.get_dummies(y))
### Label Encoder
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
labelencoder=LabelEncoder()
y=to_categorical(labelencoder.fit_transform(y))

In [44]:
y.shape

(21952, 28)

In [45]:
### Train Test Split
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=0)

In [46]:
y_train.shape

(17561, 28)

In [47]:
X_train.shape

(17561, 40)

In [48]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout,Activation,Flatten
from tensorflow.keras.optimizers import Adam
from sklearn import metrics

In [49]:
import librosa
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import os
from PIL import Image
import pathlib
from keras import layers
from keras import layers
import keras

In [50]:
model = Sequential()
model.add(layers.Dense(256, activation='relu', input_shape=(X_train.shape[1],)))
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(28, activation='softmax'))
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [51]:
model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_20 (Dense)            (None, 256)               10496     
                                                                 
 dense_21 (Dense)            (None, 128)               32896     
                                                                 
 dense_22 (Dense)            (None, 64)                8256      
                                                                 
 dense_23 (Dense)            (None, 28)                1820      
                                                                 
Total params: 53,468
Trainable params: 53,468
Non-trainable params: 0
_________________________________________________________________


In [53]:
## Trianing my model
from tensorflow.keras.callbacks import ModelCheckpoint
from datetime import datetime 

num_epochs = 40
num_batch_size = 32

checkpointer = ModelCheckpoint(filepath='audio_classification.hdf5', 
                               verbose=1, save_best_only=True)
start = datetime.now()

model.fit(X_train, y_train, batch_size=num_batch_size, epochs=num_epochs, validation_data=(X_test, y_test), callbacks=[checkpointer], verbose=1)


duration = datetime.now() - start
print("Training completed in time: ", duration)

Epoch 1/40
547/549 [============================>.] - ETA: 0s - loss: 0.0462 - accuracy: 0.9861
Epoch 1: val_loss improved from inf to 0.10486, saving model to audio_classification.hdf5
549/549 [==============================] - 2s 4ms/step - loss: 0.0461 - accuracy: 0.9861 - val_loss: 0.1049 - val_accuracy: 0.9697
Epoch 2/40
537/549 [============================>.] - ETA: 0s - loss: 0.0643 - accuracy: 0.9780
Epoch 2: val_loss improved from 0.10486 to 0.08582, saving model to audio_classification.hdf5
549/549 [==============================] - 2s 4ms/step - loss: 0.0657 - accuracy: 0.9776 - val_loss: 0.0858 - val_accuracy: 0.9722
Epoch 3/40
540/549 [============================>.] - ETA: 0s - loss: 0.0984 - accuracy: 0.9686
Epoch 3: val_loss did not improve from 0.08582
549/549 [==============================] - 2s 4ms/step - loss: 0.0973 - accuracy: 0.9689 - val_loss: 0.1047 - val_accuracy: 0.9702
Epoch 4/40
549/549 [==============================] - ETA: 0s - loss: 0.0494 - accuracy:

In [56]:
filename="007.wav"
audio, sample_rate = librosa.load(filename, res_type='kaiser_fast') 
mfccs_features = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
mfccs_scaled_features = np.mean(mfccs_features.T,axis=0)

print(mfccs_scaled_features)
mfccs_scaled_features=mfccs_scaled_features.reshape(1,-1)
print(mfccs_scaled_features)
print(mfccs_scaled_features.shape)
predicted_label=model.predict(mfccs_scaled_features)
print(predicted_label)
# prediction_class = labelencoder.inverse_transform(predicted_label) 
# prediction_class
y_classes = np.argmax(predicted_label, axis=-1)
print(y_classes)

[-196.17476     186.60579     -43.693718     -2.95328     -22.635359
   -1.6014059     5.051543     14.187795    -20.004782      4.884835
   -3.197479    -11.130185     -0.4112453    -8.425037      7.6617155
  -21.629713     -2.4375868    -8.845539     -3.4667387    -0.76815015
  -13.189546      6.372229    -14.55708       1.0147884    -4.0114837
  -10.856888     -2.2666636    -7.5505314    -4.776728     -8.244527
   -1.9299394    -5.496899     -4.0487366    -3.146125      1.968653
    3.094914      7.056366     16.88408      14.446403     18.89331   ]
[[-196.17476     186.60579     -43.693718     -2.95328     -22.635359
    -1.6014059     5.051543     14.187795    -20.004782      4.884835
    -3.197479    -11.130185     -0.4112453    -8.425037      7.6617155
   -21.629713     -2.4375868    -8.845539     -3.4667387    -0.76815015
   -13.189546      6.372229    -14.55708       1.0147884    -4.0114837
   -10.856888     -2.2666636    -7.5505314    -4.776728     -8.244527
    -1.9299394   

In [57]:
data['mapping'][8]

'Ar-Rahmaan'